<a href="https://colab.research.google.com/github/SohaHussain/Neural-Networks-and-Deep-Learning/blob/main/image_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# loading data

In [3]:
!gdown --id 1onaG42NZft3wCE1WH0GDEbUhu75fedP5

Downloading...
From: https://drive.google.com/uc?id=1onaG42NZft3wCE1WH0GDEbUhu75fedP5
To: /content/horse-or-human.zip
100% 150M/150M [00:01<00:00, 102MB/s] 


In [20]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from google.colab import files
from keras.preprocessing import image

In [5]:
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

In [6]:
train_horse_dir = os.path.join('./horse-or-human/horses')
train_human_dir = os.path.join('./horse-or-human/humans')

In [7]:
train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:5])
train_human_names = os.listdir(train_human_dir)
print(train_human_names[:5])

['horse12-7.png', 'horse07-9.png', 'horse42-2.png', 'horse49-2.png', 'horse08-6.png']
['human17-22.png', 'human03-19.png', 'human14-01.png', 'human07-20.png', 'human08-19.png']


In [8]:
print("total training horse images",len(os.listdir(train_horse_dir)))
print("total training human images",len(os.listdir(train_human_dir)))

total training horse images 500
total training human images 527


# builiding model

In [13]:
model = tf.keras.models.Sequential([
                                    #first convolution
                                    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    #second convolution
                                    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    #third convolution
                                    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    #fourth convolution
                                    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    #fifth convolution
                                    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    #flatten
                                    tf.keras.layers.Flatten(),
                                    #512 neuron hidden layer
                                    tf.keras.layers.Dense(512,activation='relu'),
                                    #output layer
                                    tf.keras.layers.Dense(1,activation='sigmoid')

])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [16]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

# data preprocessing

Let's set up data generators that will read pictures in our source folders, convert them to float32 tensors, and feed them (with their labels) to our network. We'll have one generator for the training images and one for the validation images. Our generators will yield batches of images of size 300x300 and their labels (binary).

As you may already know, data that goes into neural networks should usually be normalized in some way to make it more amenable to processing by the network. (It is uncommon to feed raw pixels into a convnet.) In our case, we will preprocess our images by normalizing the pixel values to be in the [0, 1] range (originally all values are in the [0, 255] range).

In Keras this can be done via the keras.preprocessing.image.ImageDataGenerator class using the rescale parameter. This ImageDataGenerator class allows you to instantiate generators of augmented image batches (and their labels) via .flow(data, labels) or .flow_from_directory(directory). These generators can then be used with the Keras model methods that accept data generators as inputs: fit, evaluate_generator, and predict_generator.

In [18]:
traindatagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = traindatagen.flow_from_directory(
        './horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')


Found 1027 images belonging to 2 classes.


#training the model

In [19]:
train = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1
)

Epoch 1/15
8/8 [==============================] - 87s 12s/step - loss: 0.9215 - accuracy: 0.5539
Epoch 2/15
8/8 [==============================] - 81s 10s/step - loss: 0.7329 - accuracy: 0.5673
Epoch 3/15
8/8 [==============================] - 80s 10s/step - loss: 0.5916 - accuracy: 0.6641
Epoch 4/15
8/8 [==============================] - 81s 10s/step - loss: 0.4767 - accuracy: 0.8242
Epoch 5/15
8/8 [==============================] - 81s 10s/step - loss: 0.2181 - accuracy: 0.9210
Epoch 6/15
8/8 [==============================] - 91s 11s/step - loss: 0.3254 - accuracy: 0.8867
Epoch 7/15
8/8 [==============================] - 90s 11s/step - loss: 0.2082 - accuracy: 0.9092
Epoch 8/15
8/8 [==============================] - 81s 11s/step - loss: 0.1967 - accuracy: 0.9344
Epoch 9/15
8/8 [==============================] - 79s 10s/step - loss: 0.2507 - accuracy: 0.9099
Epoch 10/15
8/8 [==============================] - 81s 10s/step - loss: 0.0513 - accuracy: 0.9833
Epoch 11/15
8/8 [============

# running the model

In [21]:
upload = files.upload()
for fn in upload.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving download1.jpg to download1.jpg
[1.5511754e-33]
download1.jpg is a horse
